# 2.数据预处理
为了能⽤深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，⽽不是从那些准备好的张量格式数据开始，常用数据处理工具有numpy,pandas等

## 2.2.1 读取数据集
举⼀个例⼦，我们⾸先创建⼀个⼈⼯数据集，并存储在CSV（逗号分隔值）⽂件 ../data/house_tiny.csv中。以其他格式存储的数据也可以通过类似的⽅式进⾏处理。下⾯我们将数据集按⾏写⼊CSV⽂件中。

In [1]:
import os

os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file = os.path.join('..','data','house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms, Alley, Prices\n') # 列名
    f.write('NA,Pave,127500\n') # 没行表示一个样本
    f.write('1,Pave,124500\n') 
    f.write('3,NA,123200\n') 
    f.write('NA,NA,123200\n') 

要从创建的CSV⽂件中加载原始数据集，我们导⼊pandas包并调⽤read_csv函数。该数据集有四⾏三列。其
中每⾏描述了房间数量（“NumRooms”）、巷⼦类型（“Alley”）和房屋价格（“Price”）。

In [4]:
import pandas as pd

print('data dir',data_file)
data = pd.read_csv(data_file)
print(data)

data dir ..\data\house_tiny.csv
   NumRooms  Alley   Prices
0       NaN   Pave   127500
1       1.0   Pave   124500
2       3.0    NaN   123200
3       NaN    NaN   123200
